In [1]:
import numpy as np
import h5py

hf_x = h5py.File('../data/logspec_x.h5', 'r')
x_train = hf_x.get('x_train_data')[()]
x_train = x_train.reshape((64727, 99, 81))
x_train = np.stack((x_train,) * 3, axis=-1)

hf_y = h5py.File('../data/logspec_y.h5', 'r')
y_train = hf_y.get('y_train_data')[()]
y_train = np.argmax(np.array(y_train), axis=1)

(x_train.shape, y_train.shape)

((64727, 99, 81, 3), (64727,))

In [2]:
from tensorflow.keras.layers import GRU, GlobalMaxPooling2D, Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, Conv3D, ConvLSTM2D
from tensorflow.keras.models import Sequential, Model
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50

NUM_CLASSES = 12
resnet = ResNet50(include_top=False, weights='imagenet', input_shape=(99, 81, 3))

# making all the layers trainable
for layer in resnet.layers:
    layer.trainable = True

out = GlobalMaxPooling2D()(resnet.output)
out = Dense(NUM_CLASSES, activation='softmax')(out)
model = Model(resnet.input, out)
model.summary()

/opt/anaconda3/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 99, 81, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 105, 87, 3)   0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 50, 41, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalizationV1) (None, 50, 41, 64)   256         conv1[0][0]                      
______________________________________________________________________________________________

In [3]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['categorical_accuracy'])
epochs = 3
batch_size = 32
x_t, x_v, y_t, y_v = train_test_split(x_train, y_train, test_size=0.1, random_state=2017)
history = model.fit(x_t, y_t,
                    validation_data=[x_v, y_v], 
                    epochs=epochs,
                    batch_size=batch_size,
                    shuffle=True,verbose=1)

MemoryError: 